### Neshyba, 2024

# Visualizing $G(T,P)$

## A reminder about $S(T,P)$
This is just a reminder about how we got $S(T,P)$ back in *VisualizingSTP* (because we're going to use the same techniques to get $G(T,P)$). The basic idea is that the slopes $S(T,P)$ are given by

$$
\big (\partial S/ \partial T \big )_P = {C_P \over T} \ \ \ \ (1)
$$ 

and 

$$
\big (\partial S/ \partial P \big )_T = -\alpha_P V \ \ \ \ (2)
$$ 

In *VisualizingSTP*, you used PL.Integrator to integrate $dS = \big (\partial S/ \partial T \big )_P dT + \big (\partial S/ \partial P \big )_T dP$ to produce surfaces like the ones shown in Fig. 1.

<p style='text-align: center;'>
<img src="http://webspace.pugetsound.edu/facultypages/nesh/Notebook/STP for gas and liquid.png" height="500" width="500"/>  
<p style='text-align: center;'>
<strong>Figure 1</strong>. Entropy of a liquid and a gas, in a temperature-pressure state space.
</p>

Doing so required that we know the heat capacity ($C_P$), isobaric expansivity ($\alpha_P$), and volume ($V$) of the liquid and gas. In general those are temperature- and pressure-dependent, but there are some approximations we'll make here that will simplify matters:

- For water vapor, use of an ideal gas assumption leads to $\alpha_P  V \approx {R \over P}$.

- For liquid water, we'll assume $\alpha_P$ and $V$ to be constants, values for which one can find online (e.g., https://webbook.nist.gov/chemistry/name-ser/). 

PL.Integrator also required standard-state entropies of formation, $S_f^o$ (also available at https://webbook.nist.gov/chemistry/name-ser/).


## Calculating $G(T,P)$ from its differential equation of state
Here, you'll be constructing $G(T,P)$ for liquid and gaseous phases of water, using the same methods: by integrating the differential equation of state for the Gibbs energy,

$$
dG = -S dT + V dP \ \ \ \ (3)
$$

To carry out this integration, you'll need the slopes of $G(T,P)$, which (according to Eq. 3) are $S(T,P)$ and $V(T,P)$, as well as the Standard-state Gibbs energies of formation (from https://webbook.nist.gov/chemistry/name-ser/). The result should look something like the surfaces in Fig. 2.

<p style='text-align: center;'>
<img src="http://webspace.pugetsound.edu/facultypages/nesh/Notebook/GTP for gas and liquid.png" height="500" width="500"/>  
<p style='text-align: center;'>
<strong>Figure 2</strong>. Gibbs energy of a liquid and a gas, in a temperature-pressure state space.
</p>


## Connection to *Clausius-Clapeyron*
The line of intersection between the Gibbs energy of liquid and gaseous water shown in Fig. 2 is pretty important: it is the set of points at which liquid and gas are in equilibrium. One big take-home of this exercise is that you'll make the connection between this intersection and the phase boundaries in phase diagrams that we've worked with before -- to be precise, the Clausius-Clapeyron liquid/gas phase boundary. 

## Intensive, not extensive
We're going to leave out the $n$ in our formulas for this exercise, treating all variables here as intensive quantities. So, the heat capacities appearing in the above are all molar heat capacities, and the volumes are molar volumes.

## Learning Goals
- I know how to numerially construct Gibbs energy state functions as thermodynamic surfaces, and how to display those surfaces using *PL.plot_surfaces*.
- I can identify where phase boundaries appear in graphs of Gibbs surfaces of vapor and liquid.
- I can predict what the Gibbs energy thermodynamic surface of any substance will look like in a temperature-pressure state space, based on general features of the differential equation of state for $G(T,P)$.

In [ ]:
# Import resources
import pint; from pint import UnitRegistry; AssignQuantity = UnitRegistry().Quantity
import warnings; warnings.filterwarnings("ignore", "The unit of the quantity is stripped when downcasting to ndarray")
import numpy as np
import sys; sys.path.append('/home'); import PchemLibrary as PL

### $T,P$ state space and constants
In the cell below, we lay out a $T,P$ state space, with units.

In [ ]:
# Laying out the state space
Tgrid,Pgrid = PL.Statespace([273,390,60],[.01,1.5,101])
Tgrid = AssignQuantity(Tgrid,'K')
Pgrid = AssignQuantity(Pgrid,'bar')

# Gas constant in SI units
R = AssignQuantity(8.314,'J /mol/K')

# Standard state conditions
T0 = AssignQuantity(298,'K')
P0 = AssignQuantity(1,'bar')

### Constants pertaining to water
The following cell provides constants for water you'll need to calculate the entropy surfaces - see https://webbook.nist.gov/chemistry/name-ser/. All are given as intensive quantities.

In [ ]:
# Parameters for gaseous water
V_gas = R*Tgrid/Pgrid
CP_gas = AssignQuantity(33.6,'J/K/mol')
SS_gas = AssignQuantity(188.835,'J/K/mol') # Standard-state entropy at 

# Parameters for liquid water
V_liq = AssignQuantity(0.018,'L/mole')*np.ones(np.shape(Tgrid))
alpha_P_liq = AssignQuantity(2.1e-4,'1/K')
CP_liq = AssignQuantity(75.4,'J/K/mol')
SS_liq = AssignQuantity(69.95,'J/K/mol')

### Visuaizing $S(T,P)$ 
Below, we recall how we constructed and visualized $S(T,P)$ for for gaseous and liquid water.

In [ ]:
# (dS/dT)_P for the gas; call the result dSdT_gas
dSdT_gas = CP_gas/Tgrid

# (dS/dT)_P for the liquid; call the result dSdT_liq
dSdT_liq = CP_liq/Tgrid

# dSdP for the gas; call the result dSdP_gas
dSdP_gas = -R/Pgrid

# dSdP for the liquid; call the result dSdP_liq
dSdP_liq = -V_liq*alpha_P_liq

# Calculate the entropy of the gas and liquid as thermodynamic surfaces
S_gas = PL.Integrator([Tgrid, Pgrid], dSdT_gas, dSdP_gas, AssignQuantity, Units='J/K/mol', SState=[T0,P0,SS_gas])
S_liq = PL.Integrator([Tgrid, Pgrid], dSdT_liq, dSdP_liq, AssignQuantity, Units='J/K/mol', SState=[T0,P0,SS_liq])

# Putting both surfaces we want to plot into a Python list
Slist = [S_gas, S_liq]

# Prepping the labels
xlabel = "T "+str(Tgrid.units)
ylabel = "P "+str(Pgrid.units)
zlabel = "S "+str(S_liq.units)
llist = [xlabel,ylabel,zlabel] # There are three elements in this list because we have x, y, and z axes

# Prepping the colors
clist = ['green', 'blue'] # There are two elements in this list because we are displaying two surfaces
tlist = ['gas', 'liq'] # Also two elements in this list, for the same reason

# Plotting the entropies using PL.plot_surfaces
PL.plot_surfaces(Tgrid,Pgrid,Slist,colorlist=clist,titlelist=tlist,labellist=llist)

### Integrating the differential equation of state for the Gibbs energy of the gas
In the cell below, you have two tasks:
1. Use PL.Integrator to construct the thermodynamic Gibbs energy surface of water vapor, $G_{gas}(T,P)$, by integrating Eq. 3, specifying units of ${kJ \over mol}$.
1. Do the same for $G_{liq}(T,P)$. 
1. Use PL.plot_surfaces to plot $G_{gas}(T,P)$ and $G_{liq}(T,P)$ together.

In [ ]:
# Your code here 


### Pause for analysis
If you orient the view so that you're looking down on the surface, with temperature increasing left-to-right, and pressure increasing bottom-to-top, you'll be able to "see" the Clausius-Clapeyron equation in action.

### Refresh/save/validate/close/submit/logout